In [17]:
# json to df

import pandas as pd
from transformers import AutoTokenizer, AutoConfig,AutoModelForMaskedLM, AutoModelForSequenceClassification, Trainer, TrainingArguments, RobertaConfig, RobertaTokenizer, RobertaForSequenceClassification, BertTokenizer

#csv로드 후 str to dict 해준다.
df = pd.read_csv("/opt/ml/dataset/train/train.csv")
df["subject_entity"] = df["subject_entity"].apply(lambda x : eval(x))
df["object_entity"] = df["object_entity"].apply(lambda x : eval(x))

for key in ["word", "type"]:
    df["subject_" + key] =  df["subject_entity"].apply(lambda x : x[key])
    df["object_" + key] =  df["object_entity"].apply(lambda x : x[key])

for key in ["start_idx", "end_idx"]:
    df["subject_" + key] =  df["subject_entity"].apply(lambda x : x[key])

for key in ["start_idx", "end_idx"]:
    df["object_" + key] =  df["object_entity"].apply(lambda x : x[key])

In [18]:
# 5분의 1 데이터가 문제.
duplicated_df = df[df.duplicated(["sentence"], keep=False)]

print(len(duplicated_df))

7090


In [21]:
# 동일 문장에서 여러 관계가 존재한다면 여러개가 나올 수 있다.
duplicated_df.sort_values(by =["sentence"], axis =0).head(10)

,id,sentence,subject_entity,object_entity,label,source,subject_word,object_word,subject_type,object_type,subject_start_idx,subject_end_idx,object_start_idx,object_end_idx
919,919,"""2006년 지방선거에 참패하고 민주당과 그 전신인 열린 우리당은 노 씨의 국정 운...","{'word': '열린 우리당', 'start_idx': 29, 'end_idx':...","{'word': '노무현', 'start_idx': 127, 'end_idx': 1...",no_relation,wikipedia,열린 우리당,노무현,ORG,PER,29,34,127,129
14736,14736,"""2006년 지방선거에 참패하고 민주당과 그 전신인 열린 우리당은 노 씨의 국정 운...","{'word': '열린 우리당', 'start_idx': 29, 'end_idx':...","{'word': '정세균', 'start_idx': 108, 'end_idx': 1...",no_relation,wikipedia,열린 우리당,정세균,ORG,PER,29,34,108,110
28051,28051,"""소이현, 인교진이 하는 광고는 괜찮고"", ""차 광고할 때 여자가 남자한테 하던 소...","{'word': '소이현', 'start_idx': 52, 'end_idx': 54...","{'word': '인교진', 'start_idx': 6, 'end_idx': 8, ...",no_relation,wikitree,소이현,인교진,PER,PER,52,54,6,8
3588,3588,"""소이현, 인교진이 하는 광고는 괜찮고"", ""차 광고할 때 여자가 남자한테 하던 소...","{'word': '인교진', 'start_idx': 6, 'end_idx': 8, ...","{'word': '소이현', 'start_idx': 52, 'end_idx': 54...",no_relation,wikitree,인교진,소이현,PER,PER,6,8,52,54
32434,32434,"""탕약망""이란 예수회 선교사 천문학자 아담 샬의 중국 이름이다.","{'word': '아담 샬', 'start_idx': 21, 'end_idx': 2...","{'word': '예수회', 'start_idx': 8, 'end_idx': 10,...",per:employee_of,wikipedia,아담 샬,예수회,PER,ORG,21,24,8,10
19427,19427,"""탕약망""이란 예수회 선교사 천문학자 아담 샬의 중국 이름이다.","{'word': '탕약망', 'start_idx': 1, 'end_idx': 3, ...","{'word': '예수회', 'start_idx': 8, 'end_idx': 10,...",per:employee_of,wikipedia,탕약망,예수회,PER,ORG,1,3,8,10
11401,11401,"'2019 코리아 스타트업 테크쇼'는 경상남도, 창원시, 경남지방중소벤처기업청이 공...","{'word': '경상남도', 'start_idx': 21, 'end_idx': 2...","{'word': '창원시', 'start_idx': 27, 'end_idx': 29...",org:members,wikitree,경상남도,창원시,ORG,ORG,21,24,27,29
29166,29166,"'2019 코리아 스타트업 테크쇼'는 경상남도, 창원시, 경남지방중소벤처기업청이 공...","{'word': '창원시', 'start_idx': 27, 'end_idx': 29...","{'word': '경상남도', 'start_idx': 21, 'end_idx': 2...",org:place_of_headquarters,wikitree,창원시,경상남도,ORG,ORG,27,29,21,24
23316,23316,"'Project Fairy(프로젝트 페어리)'라는 유닛명으로 가나하 히비키, 시죠 ...","{'word': '가나하 히비키', 'start_idx': 34, 'end_idx'...","{'word': '프로젝트 페어리', 'start_idx': 15, 'end_idx...",per:employee_of,wikipedia,가나하 히비키,프로젝트 페어리,PER,ORG,34,40,15,22
16566,16566,"'Project Fairy(프로젝트 페어리)'라는 유닛명으로 가나하 히비키, 시죠 ...","{'word': '시죠 타카네', 'start_idx': 43, 'end_idx':...","{'word': '프로젝트 페어리', 'start_idx': 15, 'end_idx...",per:employee_of,wikipedia,시죠 타카네,프로젝트 페어리,PER,ORG,43,48,15,22


In [26]:
duplicated_df_label = df[df.duplicated(["sentence", "subject_word", "object_word"], keep=False)]
duplicated_df.sort_values(by =["sentence"], axis =0)
print(len(duplicated_df_label))
duplicated_df_no_label = df[df.duplicated(["sentence", "subject_word", "object_word", "label"], keep=False)]
print(len(duplicated_df_no_label))


105
92


In [31]:
troble = list(set(list(duplicated_df_label.index)) - set(list(duplicated_df_no_label.index)))

In [36]:
df.loc[troble].sort_values(by =["sentence"], axis =0)

,id,sentence,subject_entity,object_entity,label,source,subject_word,object_word,subject_type,object_type,subject_start_idx,subject_end_idx,object_start_idx,object_end_idx
13496,13496,"2015년 11월 조현아 전 대한항공 부사장, 조원태 대한항공 부사장, 조현민 대한...","{'word': '대한항공', 'start_idx': 16, 'end_idx': 1...","{'word': '조양호', 'start_idx': 54, 'end_idx': 56...",org:top_members/employees,wikipedia,대한항공,조양호,ORG,PER,16,19,54,56
18458,18458,"2015년 11월 조현아 전 대한항공 부사장, 조원태 대한항공 부사장, 조현민 대한...","{'word': '대한항공', 'start_idx': 44, 'end_idx': 4...","{'word': '조양호', 'start_idx': 54, 'end_idx': 56...",no_relation,wikipedia,대한항공,조양호,ORG,PER,44,47,54,56
6749,6749,대한항공은 5일 조양호 회장의 3자녀가 보유한 싸이버스카이 주식 9만9900주 전량...,"{'word': '대한항공', 'start_idx': 0, 'end_idx': 3,...","{'word': '조양호', 'start_idx': 9, 'end_idx': 11,...",no_relation,wikipedia,대한항공,조양호,ORG,PER,0,3,9,11
12829,12829,대한항공은 5일 조양호 회장의 3자녀가 보유한 싸이버스카이 주식 9만9900주 전량...,"{'word': '대한항공', 'start_idx': 0, 'end_idx': 3,...","{'word': '조양호', 'start_idx': 9, 'end_idx': 11,...",org:top_members/employees,wikipedia,대한항공,조양호,ORG,PER,0,3,9,11
32299,32299,배우 김병철 씨가 연기하는 정복동은 천리마마트를 망하게 하기 위해 여러 계획을 세우...,"{'word': '정복동', 'start_idx': 15, 'end_idx': 17...","{'word': '김병철', 'start_idx': 3, 'end_idx': 5, ...",per:alternate_names,wikitree,정복동,김병철,PER,PER,15,17,3,5
8364,8364,배우 김병철 씨가 연기하는 정복동은 천리마마트를 망하게 하기 위해 여러 계획을 세우...,"{'word': '정복동', 'start_idx': 15, 'end_idx': 17...","{'word': '김병철', 'start_idx': 3, 'end_idx': 5, ...",no_relation,wikitree,정복동,김병철,PER,PER,15,17,3,5
22258,22258,"영화 '버즈 오브 프레이'는 배트맨이 없는 고담시에서 할리퀸, 헌트리스, 블랙 카나...","{'word': '배트맨', 'start_idx': 16, 'end_idx': 18...","{'word': '고담시', 'start_idx': 24, 'end_idx': 26...",no_relation,wikitree,배트맨,고담시,PER,LOC,16,18,24,26
11511,11511,"영화 '버즈 오브 프레이'는 배트맨이 없는 고담시에서 할리퀸, 헌트리스, 블랙 카나...","{'word': '배트맨', 'start_idx': 16, 'end_idx': 18...","{'word': '고담시', 'start_idx': 24, 'end_idx': 26...",per:place_of_residence,wikitree,배트맨,고담시,PER,LOC,16,18,24,26
3296,3296,"이날 프로그램 공개에서는 전북영산작법보존회와 김명신‧정상희의 사제동행 판소리, 관악...","{'word': '강태환', 'start_idx': 62, 'end_idx': 64...","{'word': '색소폰', 'start_idx': 58, 'end_idx': 60...",per:title,wikitree,강태환,색소폰,PER,POH,62,64,58,60
32107,32107,"이용빈 후보는 전)문재인 대통령후보 국민주권광주선대위 공동선대위원장, 전)20대 국...","{'word': '문재인', 'start_idx': 10, 'end_idx': 12...","{'word': '더불어민주당', 'start_idx': 134, 'end_idx'...",per:employee_of,wikitree,문재인,더불어민주당,PER,ORG,10,12,134,139


In [11]:
# test data 중복처리
df = pd.read_csv("/opt/ml/dataset/test/test_data.csv")

duplicated_df = df[df.duplicated(["sentence"], keep=False)]

print(len(duplicated_df))


2
